In [4]:
# import transformers
# import torch
# import os
# from transformers import (
#     AutoModelForCausalLM,
#     AutoTokenizer,
#     BitsAndBytesConfig,
#     HfArgumentParser,
#     TrainingArguments,
#     pipeline,
#     logging,
# )


# # Hugging face repo name
# model = "NousResearch/llama-2-7b-chat-hf"

# tokenizer = AutoTokenizer.from_pretrained(model)

# pipeline = transformers.pipeline(
#     "text-generation",
#     model=model,
#     torch_dtype=torch.float16,
#     device_map="auto" # if you have GPU
# )

# sequences = pipeline(
#     'I liked "Breaking Bad" and "Band of Brothers". Do you have any recommendations of other shows I might like?\n',
#     do_sample=True,
#     top_k=10,
#     top_p = 0.9,
#     temperature = 0.2,
#     num_return_sequences=1,
#     eos_token_id=tokenizer.eos_token_id,
#     max_length=200, # can increase the length of sequence
# )
# for seq in sequences:
#     print(f"Result: {seq['generated_text']}")

In [5]:
!pip3 install -q accelerate==0.21.0 bitsandbytes==0.40.2 transformers==4.31.0


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 41.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 52.7 MB/s eta 0:00:00


In [6]:
import json
import transformers
import torch
import os
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)

In [7]:
# Load model and tokenizer
model_name = "NousResearch/llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

In [8]:
model_pipeline = transformers.pipeline(
    "text-generation",
    model=model_name,
    torch_dtype=torch.float16,
    device_map="auto"  # Use GPU if available
)


config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/179 [00:00<?, ?B/s]

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [10]:
# from datasets import load_dataset
import pandas as pd
import json
import random

dataset = pd.read_csv("data_full_v1.csv")
dataset['ground_truth'] = dataset.apply(lambda row: 1 if row['label']=='yes' else 0, axis=1)

dataset = dataset.sample(frac=1)

# dataset[0:1000]['ground_truth'].value_counts()
dataset = json.loads(dataset.to_json(orient ='records'))


In [11]:
# Format the COPA query for the GPT-3.5 Turbo model
def format_query(item):
  return f"{item['prompt']}. Yes or No?"

  # return f"{item['prompt']}. Output only single digit 0 or 1. 0 for no and 1 for yes. Output should have only single digit. Do not output any text other than a single digit"

format_query(dataset[0])

'Imagine a self-contained, hypothetical world with only the following conditions, and without any unmentioned factors or causal relationships: Eating citrus has a direct effect on vitmain C. Vitmain C has a direct effect on curly hair.\nThe overall probability of citrus intake is 92%. The probability of absence of citrus and curly hair is 5%. The probability of citrus intake and curly hair is 38%.\nIs the chance of curly hair larger when observing citrus intake?. Yes or No?'

In [12]:
# # response = query_gpt3_5_turbo(format_query(dataset[0]))
# response = model_pipeline(format_query(dataset[0]), do_sample=True, top_k=10, top_p=0.9, temperature=0.2, num_return_sequences=1, eos_token_id=tokenizer.eos_token_id, max_length=500)
# generated_text = response[0]['generated_text']
# print(generated_text)
# # print(generated_text[-1])

# generated_text = generated_text.removeprefix(format_query(dataset[0]))

# if 'yes' in generated_text.lower() and (not 'no' in generated_text.lower()):
#   print(1)
# elif 'no' in generated_text.lower() and (not 'yes' in generated_text.lower()):
#   print(0)
# else:
#   raise Exception("failure")

In [15]:
from tenacity import retry, stop_after_attempt, wait_random_exponential, RetryError
import re

correct_count = 0
processed_items = 0
processed = []
failed_items = []
y_true = []
y_pred = []

# Evaluate the accuracy of GPT-3.5 Turbo on the COPA dataset
@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(1))
def evaluate_accuracy(item):
      global correct_count
      query = format_query(item)
      # print(query)
      response = model_pipeline(query, do_sample=True, top_k=10, top_p=0.9, temperature=0.2, num_return_sequences=1, eos_token_id=tokenizer.eos_token_id, max_length=500)

      generated_text = response[0]['generated_text']
      # print(generated_text)
      generated_text = generated_text.removeprefix(query).lower()

      # print(generated_text)
      generated_text = [w for w in re.split('\W', generated_text) if w]
      # generated_text = ''.join(filter(str.isalnum, generated_text)).split()

      if (('yes' in generated_text) or ('yes.' in generated_text)) and ((not 'no' in generated_text) and (not 'no.' in generated_text)):
        response = 1
      elif (('no' in generated_text) or ('no.' in generated_text)) and ((not 'yes' in generated_text) and (not 'yes.' in generated_text)):
        response = 0
      else:
        print(generated_text)
        raise Exception("failure")

      # response = pred[-1]
      print(response)
      # print(item['ground_truth'])
      correct_hypothesis = ((int(response) == item['ground_truth'])==True)
      y_true.append(item['ground_truth'])
      y_pred.append(int(response))
      # print(correct_hypothesis)
      correct_count += correct_hypothesis
      print("correct_count is ", correct_count)
      # print("completed\n")


# Evaluate accuracy
def evaluate_accuracy_wrapper(data):
      global correct_count
      global processed_items
      global failed_items
      for item in data:
          try:
              evaluate_accuracy(item)
              processed_items = processed_items +1
              processed.append(item)
          except RetryError as e:
              print("failed")
              failed_items.append(item)
          if processed_items==1000:
            break
          if processed_items%10==0:
            print('processed_items ', processed_items)
      print("correct_count in wrapper is ", correct_count)
      return correct_count / len(data)

accuracy = evaluate_accuracy_wrapper(dataset)
print(f"Accuracy: {accuracy * 100:.2f}%")

0
correct_count is  1
1
correct_count is  1
1
correct_count is  2
1
correct_count is  3
1
correct_count is  4
0
correct_count is  5
1
correct_count is  6
['hint', 'use', 'bayes', 'theorem']
failed
1
correct_count is  7
1
correct_count is  7
1
correct_count is  8
processed_items  10
[]
failed
processed_items  10
['please', 'answer', 'with', 'a', 'simple', 'yes', 'or', 'no']
failed
processed_items  10
1
correct_count is  8
1
correct_count is  8
1
correct_count is  8
1
correct_count is  8
['hint', 'use', 'the', 'information', 'provided', 'in', 'the', 'passage', 'to', 'answer', 'the', 'question']
failed
['please', 'provide', 'a', 'clear', 'and', 'concise', 'answer', 'to', 'the', 'question', 'and', 'explain', 'your', 'reasoning']
failed
1
correct_count is  9
1
correct_count is  10
['note', 'the', 'given', 'conditions', 'are', 'a', 'simplified', 'example', 'and', 'in', 'reality', 'there', 'may', 'be', 'many', 'other', 'factors', 'that', 'affect', 'the', 'probability', 'of', 'kwoz']
failed
1


In [16]:
# y_true
processed

[{'id': 13701,
  'prompt': 'Imagine a self-contained, hypothetical world with only the following conditions, and without any unmentioned factors or causal relationships: Eating citrus has a direct effect on vitmain C. Vitmain C has a direct effect on curly hair.\nThe overall probability of citrus intake is 92%. The probability of absence of citrus and curly hair is 5%. The probability of citrus intake and curly hair is 38%.\nIs the chance of curly hair larger when observing citrus intake?',
  'label': 'no',
  'ground_truth': 0},
 {'id': 29649,
  'prompt': 'Imagine a self-contained, hypothetical world with only the following conditions, and without any unmentioned factors or causal relationships: Swoy has a direct effect on rixq and xevu. Zuph has a direct effect on rixq. Rixq has a direct effect on xevu. Swoy is unobserved.\nWe know that swoy and zuph causes rixq. swoy or rixq causes xevu. We observed an individual is not zuph and an individual is not swoy.\nWould an individual is xevu

In [17]:
# y_pred

In [19]:
with open("y_true.txt", "w") as f:
    for s in y_true:
        f.write(str(s) +"\n")

In [20]:
with open("y_pred.txt", "w") as f:
    for s in y_pred:
        f.write(str(s) +"\n")

In [21]:
with open("processed.txt", "w") as f:
    for s in processed:
        f.write(str(s) +"\n")

In [18]:
from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.54      0.19      0.28       528
           1       0.47      0.82      0.60       472

    accuracy                           0.48      1000
   macro avg       0.50      0.50      0.44      1000
weighted avg       0.51      0.48      0.43      1000



In [ ]:
correct_count = 0
y_true = []
y_pred = []

def evaluate_accuracy(item):
      global correct_count
      query = format_query(item)
      # print(query)
      response = model_pipeline(format_query(item), do_sample=True, top_k=10, top_p=0.9, temperature=0.2, num_return_sequences=1, eos_token_id=tokenizer.eos_token_id, max_length=500)
      # print(response)
      generated_text = response[0]['generated_text']
      print(generated_text)
      print('\n\n')
      # print(item['ground_truth'])
      pred = generated_text.removeprefix(format_query(dataset[0]))
      # print(pred)
      response = pred[-1]
      print(response)
      correct_hypothesis = ((int(response) == item['ground_truth'])==True)
      y_true.append(item['ground_truth'])
      y_pred.append(int(response))
      # print(correct_hypothesis)
      correct_count += correct_hypothesis
      # print("correct_count is ", correct_count)
      # print("completed\n")

for index, item in enumerate(dataset[0:10]):
  if index % 100 == 0:
    print(index)
  evaluate_accuracy(item)



0
Imagine a self-contained, hypothetical world with only the following conditions, and without any unmentioned factors or causal relationships: Husband has a direct effect on wife and alarm clock. Wife has a direct effect on alarm clock.
For husbands that don't set the alarm and wives that don't set the alarm, the probability of ringing alarm is 8%. For husbands that don't set the alarm and wives that set the alarm, the probability of ringing alarm is 54%. For husbands that set the alarm and wives that don't set the alarm, the probability of ringing alarm is 41%. For husbands that set the alarm and wives that set the alarm, the probability of ringing alarm is 86%. For husbands that don't set the alarm, the probability of alarm set by wife is 74%. For husbands that set the alarm, the probability of alarm set by wife is 24%.
If we disregard the mediation effect through wife, would husband positively affect alarm clock?. Output only single digit 0 or 1. 0 for no and 1 for yes. Output shou

ValueError: invalid literal for int() with base 10: '.'

In [ ]:
y_true

[0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0]

In [ ]:
# response = query_gpt3_5_turbo(format_query(dataset[0]))
response = model_pipeline(format_query(dataset[0]), do_sample=True, top_k=10, top_p=0.9, temperature=0.2, num_return_sequences=1, eos_token_id=tokenizer.eos_token_id, max_length=500)
generated_text = response[0]['generated_text']
print(generated_text)
print(generated_text[-1])
# generated_text.removeprefix(format_query(dataset[0]))

Imagine a self-contained, hypothetical world with only the following conditions, and without any unmentioned factors or causal relationships: Husband has a direct effect on wife and alarm clock. Wife has a direct effect on alarm clock.
For husbands that don't set the alarm and wives that don't set the alarm, the probability of ringing alarm is 8%. For husbands that don't set the alarm and wives that set the alarm, the probability of ringing alarm is 54%. For husbands that set the alarm and wives that don't set the alarm, the probability of ringing alarm is 41%. For husbands that set the alarm and wives that set the alarm, the probability of ringing alarm is 86%. For husbands that don't set the alarm, the probability of alarm set by wife is 74%. For husbands that set the alarm, the probability of alarm set by wife is 24%.
If we disregard the mediation effect through wife, would husband positively affect alarm clock?. Output only single digit 0 or 1. 0 for no and 1 for yes. Output should

'.\n\nAnswer: 1'

In [ ]:
query = f"{item['premise']} {item['hypothesis1']} or {item['hypothesis2']}?"
response = model_pipeline(query, do_sample=True, top_k=10, top_p=0.9, temperature=0.2, num_return_sequences=1, eos_token_id=tokenizer.eos_token_id, max_length=200)
generated_text = response[0]['generated_text']